In [1]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image as im, ImageDraw, ImageFont
import PIL
from scipy import ndimage
from matplotlib.pyplot import figure
from glob import glob
from utils import add_margin, expand2square, put_in_a_box, binarize
from tqdm import tqdm

from typing import *

In [2]:
PATH_TO_FONT_CHARS = "../formatted_data/chars_images/chars"

PATH_TO_AUGMENTED_CHARS = "../formatted_data/chars_images/augmented_chars"
PATH_TO_ROTATED_CHARS = f"{PATH_TO_AUGMENTED_CHARS}/rotated"
PATH_TO_OFFSET_CHARS = f"{PATH_TO_AUGMENTED_CHARS}/offset"
PATH_TO_CONNECTED_CHARS = f"{PATH_TO_AUGMENTED_CHARS}/connected"
PATH_TO_RESIZED_CHARS = f"{PATH_TO_AUGMENTED_CHARS}/resized"

LOWERCASE = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя"
UPPERCASE = LOWERCASE.upper().replace("Ъ", '')

In [3]:
class Char:
    def __init__(self, path: str):
        self.path = path
        self.image = im.open(path)
        
        split = os.path.normpath(path).split(os.path.sep)
        self.name = split[-1].replace(".png", "")
        self.font = split[-2]


In [4]:
fonts = os.listdir(PATH_TO_FONT_CHARS)
chars_dict = {font: glob(f"{PATH_TO_FONT_CHARS}/{font}/*") for font in fonts}
for k, v in chars_dict.items():
    chars = []
    for char in v:
        chars.append(Char(char))
    chars_dict[k] = chars

In [5]:
charset = chars_dict['Salavat']

In [6]:
def convert_png_transparent(image, bg_color=(255,255,255)):
    array = np.array(image, dtype=np.ubyte)
    mask = (array[:,:,:3] == bg_color).all(axis=2)
    alpha = np.where(mask, 0, 255)
    array[:,:,-1] = alpha
    return im.fromarray(np.ubyte(array))

In [7]:
def get_dominant_color(img):
    palette_size = 2
    # Resize image to speed up processing
    #img = convert_png_transparent(pil_img.copy())
    img.thumbnail((100, 100))

    # Reduce colors (uses k-means internally)
    paletted = img.convert('P', palette=im.ADAPTIVE, colors=palette_size)

    # Find the color that occurs most often
    palette = paletted.getpalette()
    color_counts = sorted(paletted.getcolors(), reverse=False)
    palette_index = color_counts[0][1]
    dominant_color = palette[palette_index*3:palette_index*3+3]
    dominant_color.append(255)

    return tuple(dominant_color)

In [8]:
def expand(img, top=0, right=0, bottom=0, left=0):
    return add_margin(
             expand2square(img),
             top=top,
             bottom=bottom,
             left=left,
             right=right
           )

In [9]:
""" ROTATION FROM -15° to 15° """
for font, charset in chars_dict.items():
    font_specific_path = f"{PATH_TO_ROTATED_CHARS}/{font}"
    os.makedirs(font_specific_path, exist_ok=True)
    for char in charset:
        for degree in range(-40, 41, 5):
            char.image \
                .rotate(degree, expand=True, fillcolor=(255,255,255), resample=PIL.Image.Resampling.BICUBIC) \
                .save(f"{font_specific_path}/{char.name}_{degree}.png")


KeyboardInterrupt: 

In [ ]:
""" MOVING AROUND """
for font, charset in chars_dict.items():
    font_specific_path = f"{PATH_TO_OFFSET_CHARS}/{font}"
    os.makedirs(font_specific_path, exist_ok=True)
    for char in charset:
        for margin, side in [(m, side) for m in range(5,16,5) for side in "lrtb"]:
            if side == "l":
                image = expand(char.image, left=margin)
            elif side == "r":
                image = expand(char.image, right=margin)
            elif side == "t":
                image = expand(char.image, top=margin)
            elif side == "b":
                image = expand(char.image, bottom=margin)
            image.save(f"{font_specific_path}/{char.name}_{side}{margin}.png")

In [9]:
def euclideanDistance(coordinate1, coordinate2):
    return pow(pow(coordinate1[0] - coordinate2[0], 2) + pow(coordinate1[1] - coordinate2[1], 2), .5)

In [10]:
def closest_to(char_image, coord: Tuple[int, int]):
    bin_img = binarize(char_image)
    h, w = char_image.size
    distances = list()
    for r_idx in range(w):
        for c_idx in range(h):
            if bin_img[r_idx, c_idx] > 0.5:
                distance = euclideanDistance(coord, (r_idx, c_idx))
                distances.append((r_idx, c_idx, distance))
    return min(distances, key=lambda entry: entry[2])[:2]

# goes up and right from here
right_bottom_spot = lambda image: (closest_to(image, (image.height, image.width)), {"x": 100, "y": -30})
# goes down and right from here
right_top_spot = lambda image: (closest_to(image, (0, image.height)), {"x": 100, "y": 30})

# goes up and rigth from here
left_bottom_spot = lambda image: (closest_to(image, (image.width, 0)), {"x": 100, "y": -30})
# goes down and left from here
left_top_spot = lambda image: (closest_to(image, (0,0)), {"x": -100, "y": 30})

In [ ]:
""" CONNECTED CHARS """
for font, charset in chars_dict.items():
    font_specific_path = f"{PATH_TO_CONNECTED_CHARS}/{font}"
    os.makedirs(font_specific_path, exist_ok=True)
    for char in charset:
        dominant = get_dominant_color(char.image)
        # left_top from prev
        char_copy = char.image.copy()
        draw = ImageDraw.Draw(char_copy) 
        (y, x), destination = left_top_spot(char_copy)
        draw.line((x,y, x+destination["x"],y+destination["y"]), fill=dominant, width=2, joint="curve")     
        char_copy.save(f"{font_specific_path}/{char.name}_lt.png")
        
        # right_top to next
        char_copy = char.image.copy()
        draw = ImageDraw.Draw(char_copy)
        (y, x), destination = right_top_spot(char_copy)
        draw.line((x,y, x+destination["x"],y+destination["y"]), fill=dominant, width=2, joint="curve")
        char_copy.save(f"{font_specific_path}/{char.name}_rt.png")
        
        # right_bottom to next
        char_copy = char.image.copy()
        draw = ImageDraw.Draw(char_copy) 
        (y, x), destination = right_bottom_spot(char_copy)
        draw.line((x,y, x+destination["x"],y+destination["y"]), fill=dominant, width=2, joint="curve")
        char_copy.save(f"{font_specific_path}/{char.name}_rb.png")
        
        # left_bottom to next
        char_copy = char.image.copy()
        draw = ImageDraw.Draw(char_copy) 
        (y, x), destination = left_bottom_spot(char_copy)
        draw.line((x,y, x+destination["x"],y+destination["y"]), fill=dominant, width=2, joint="curve")
        char_copy.save(f"{font_specific_path}/{char.name}_lb.png")
        

In [ ]:
""" RESIZED, i.e. squashed or extended"""
for font, charset in chars_dict.items():
    font_specific_path = f"{PATH_TO_RESIZED_CHARS}/{font}"
    os.makedirs(font_specific_path, exist_ok=True)
    for char in charset:
        w, h = char.image.size        
        char.image.resize((w * 12 // 10, h)).save(f"{font_specific_path}/{char.name}_20%wider.png")
        char.image.resize((w * 14 // 10, h)).save(f"{font_specific_path}/{char.name}_40%wider.png")
        char.image.resize((w * 16 // 10, h)).save(f"{font_specific_path}/{char.name}_60%wider.png")
        char.image.resize((w * 18 // 10, h)).save(f"{font_specific_path}/{char.name}_80%wider.png")
        
        char.image.resize((w * 8 // 10, h)).save(f"{font_specific_path}/{char.name}_20%narrower.png")
        char.image.resize((w * 6 // 10, h)).save(f"{font_specific_path}/{char.name}_40%narrower.png")
        char.image.resize((w * 4 // 10, h)).save(f"{font_specific_path}/{char.name}_60%narrower.png")
        
        char.image.resize((w, h * 12 // 10)).save(f"{font_specific_path}/{char.name}_20%higher.png")
        char.image.resize((w, h * 14 // 10)).save(f"{font_specific_path}/{char.name}_40%higher.png")
        char.image.resize((w, h * 16 // 10)).save(f"{font_specific_path}/{char.name}_60%higher.png")
        char.image.resize((w, h * 18 // 10)).save(f"{font_specific_path}/{char.name}_80%higher.png")
        
        char.image.resize((w, h * 8 // 10)).save(f"{font_specific_path}/{char.name}_20%shorter.png")
        char.image.resize((w, h * 6 // 10)).save(f"{font_specific_path}/{char.name}_40%shorter.png")
        char.image.resize((w, h * 4 // 10)).save(f"{font_specific_path}/{char.name}_60%shorter.png")
        char.image.resize((w, h * 3 // 10)).save(f"{font_specific_path}/{char.name}_70%shorter.png")

Создаём фейковые изображения символов в связке с соседними символами и называем это умным словом "аугментация"

In [37]:
def write(font, word):
    img  = im.new("RGBA", (500,150),(255,255,255))
    ImageDraw.Draw(img).text((22, 22), word,(49, 76, 175), font=font)
    return put_in_a_box(img)

def width_of(font, word):
    return write(font, word).width

In [74]:
from copy import copy
ttfs = glob("ttfs/*.ttf")

for ttf in ttfs:
    font = ImageFont.truetype(ttf,64)
    fontname = os.path.basename(ttf).replace(".ttf", '')
    font_specific_path = f"../formatted_data/chars_images/augmented_chars/simulation/{fontname}"
    os.makedirs(font_specific_path, exist_ok=True)
    print(fontname)
    for s in tqdm(LOWERCASE):
        
        pool = copy(LOWERCASE)
        for idx in range(len(pool)):
            f = pool[idx]
            t = pool[idx // 2]
            word = f"{f}{s}{t}"
            
            img = write(font, word)
            l = img.width - width_of(font, f"{s+t}")
            r = width_of(font, f"{f+s}")
            
            s_itself = width_of(font, s)
            if s_itself - 2 <= r - l <= s_itself + 2:
                l, r = l-5, r+5
            
            img = img.crop((l, 0, r, img.height))
            img.save(f"{font_specific_path}/{word}.png")
#                 try:
#                 except Exception:
#                     None
                

Abram


100%|███████████████████████████████████████████| 33/33 [00:28<00:00,  1.15it/s]


Propisi


100%|███████████████████████████████████████████| 33/33 [00:29<00:00,  1.11it/s]


Gogol


...

100%|███████████████████████████████████████████| 33/33 [00:29<00:00,  1.13it/s]


Denistina


100%|███████████████████████████████████████████| 33/33 [00:28<00:00,  1.14it/s]


Tibalt


100%|███████████████████████████████████████████| 33/33 [00:28<00:00,  1.16it/s]


In [21]:
font = ImageFont.truetype(ttfs[0],64)
img  = im.new("RGBA", (500,100),(255,255,255))
draw = ImageDraw.Draw(img)

f,s,t = tuple("бцц")
draw.text((22, 22), f"{f}{s}{t}",(49, 76, 175),font=font)
img = put_in_a_box(img)

l, r = img.width - w[s] - w[t], img.width - w[t]
img.crop((l, 0, r, img.height))
# img